In [13]:
import pandas as pd

# Load the datasets
customers_path = 'Customers.csv'
products_path = 'Products.csv'
transactions_path = 'Transactions.csv'

customers_df = pd.read_csv(customers_path)
products_df = pd.read_csv(products_path)
transactions_df = pd.read_csv(transactions_path)

# Display the first few rows of each dataset to understand their structure
customers_df.head(), products_df.head(), transactions_df.head()


(  CustomerID        CustomerName         Region  SignupDate
 0      C0001    Lawrence Carroll  South America  2022-07-10
 1      C0002      Elizabeth Lutz           Asia  2022-02-13
 2      C0003      Michael Rivera  South America  2024-03-07
 3      C0004  Kathleen Rodriguez  South America  2022-10-09
 4      C0005         Laura Weber           Asia  2022-08-15,
   ProductID              ProductName     Category   Price
 0      P001     ActiveWear Biography        Books  169.30
 1      P002    ActiveWear Smartwatch  Electronics  346.30
 2      P003  ComfortLiving Biography        Books   44.12
 3      P004            BookWorld Rug   Home Decor   95.69
 4      P005          TechPro T-Shirt     Clothing  429.31,
   TransactionID CustomerID ProductID      TransactionDate  Quantity  \
 0        T00001      C0199      P067  2024-08-25 12:38:23         1   
 1        T00112      C0146      P067  2024-05-27 22:23:54         1   
 2        T00166      C0127      P067  2024-04-25 07:38:55    

In [14]:
# Merge Transactions with Products to include product details in the transactions
transactions_merged = pd.merge(transactions_df, products_df, on="ProductID", how="left")

# Merge the resulting data with Customers to include customer details
full_data = pd.merge(transactions_merged, customers_df, on="CustomerID", how="left")

# Display the first few rows of the combined dataset to verify
full_data.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Brittany Harvey,Asia,2024-09-04
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Kathryn Stevens,Europe,2024-04-04
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Travis Campbell,South America,2024-04-11
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Timothy Perez,Europe,2022-03-15


In [18]:
# Feature Engineering: Aggregating data at the customer level
customer_features = full_data.groupby("CustomerID").agg({
    "TotalValue": "sum",  # Total spending
    "Quantity": "sum",    # Total quantity purchased
    "Category": lambda x: x.mode()[0] if not x.mode().empty else None,  # Most purchased category
    "Region": "first",    # Customer region
    "SignupDate": "first" # Signup date
}).reset_index()

# Convert SignupDate to datetime for numerical encoding
customer_features['SignupDate'] = pd.to_datetime(customer_features['SignupDate'])

# Encode Region and Category as numerical features
customer_features = pd.get_dummies(customer_features, columns=["Region", "Category"], drop_first=True)

# Display the engineered features
customer_features.head()


,CustomerID,TotalValue,Quantity,SignupDate,Region_Europe,Region_North America,Region_South America,Category_Clothing,Category_Electronics,Category_Home Decor
0,C0001,3354.52,12,2022-07-10,False,False,True,False,True,False
1,C0002,1862.74,10,2022-02-13,False,False,False,True,False,False
2,C0003,2725.38,14,2024-03-07,False,False,True,False,False,True
3,C0004,5354.88,23,2022-10-09,False,False,True,False,False,False
4,C0005,2034.24,7,2022-08-15,False,False,False,False,True,False
